In [1]:
import numpy as np
from copy import deepcopy
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers,regularizers,metrics,optimizers
import random
import pandas as pd
from scipy.linalg import sqrtm
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import os
config=tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth=True
sess=tf.compat.v1.Session(config=config) 

In [ ]:
"""
This code is used to train VGGNet-16 using the complete CIFAR-10 dataset. 
It employs an explicit Weight Decay SGD optimizer along with WarmUp + 
Cosine Annealing Learning Rate Scheduler, and enhances the generalization 
ability of the trained network through data augmentation.
"""

In [3]:
num_classes = 10
initial_lr = 0.1
weight_decay = 1e-4
epochs = 200
warmup_epochs = 5
batch_size = 128
image_size = 32

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255
y_train_onehot=tf.keras.utils.to_categorical(y_train,num_classes=10)
y_test_onehot=tf.keras.utils.to_categorical(y_test,num_classes=10)

In [5]:
NN_16=[64,64,128,128,256,256,256,512,512,512,512,512,512]

In [6]:
def build_vggnet16(NN,input_shape=(32,32,3),num_class=10):
    model = keras.models.Sequential()
    model.add(layers.Conv2D(filters=NN[0], kernel_size=(3, 3), padding='same'
                            ,input_shape=input_shape,use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(layers.Conv2D(filters=NN[1], kernel_size=(3, 3), padding='same',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    
    #2
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(NN[2], (3, 3), padding='same',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(layers.Conv2D(NN[3], (3, 3), padding='same',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    #5
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(NN[4], (3, 3), padding='same',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(layers.Conv2D(NN[5], (3, 3), padding='same',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization()) 
    model.add(tf.keras.layers.Activation('relu'))
    model.add(layers.Conv2D(NN[6], (3, 3), padding='same',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization()) 
    model.add(tf.keras.layers.Activation('relu'))
    #10
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(NN[7], (3, 3), padding='same',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(layers.Conv2D(NN[8], (3, 3), padding='same',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(layers.Conv2D(NN[9], (3, 3), padding='same',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    #15
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(NN[10], (3, 3), padding='same',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(layers.Conv2D(NN[11], (3, 3), padding='same',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(layers.Conv2D(NN[12], (3, 3), padding='same',use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    #
    #25
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(NN[12]))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(layers.Dense(NN[12]))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(layers.Dense(num_class,activation='softmax'))
    return model
model = build_vggnet16(NN_16)

In [7]:
decay_vars = []
no_decay_vars = []
for var in model.trainable_variables:
    if 'kernel' in var.name and 'bn' not in var.name.lower():
        decay_vars.append(var)
    else:
        no_decay_vars.append(var)

In [8]:
class WarmUpCosine(tf.keras.optimizers.schedules.LearningRateSchedule):
    """
   WarmUp + Cosine Annealing Learning Rate Scheduler 
   This scheduler divides the training process into two stages:
   1) Warm-up stage (step < warmup_steps):
   The learning rate linearly increases from warmup_lr to base_lr, 
   aiming to alleviate the instability of gradients or excessive 
   parameter updates at the beginning of training, thereby enhancing 
   the stability of the training process.
   2) Cosine Annealing stage (step >= warmup_steps):
   The learning rate smoothly decays from base_lr to a value close 
   to 0 according to the cosine function, which is helpful for 
   conducting more precise parameter search in the later stage of 
   training and improving the final convergence performance. 
   """
    def __init__(self, base_lr, total_steps, warmup_steps, warmup_lr=0.0):
        """
        Parameter description:
        - base_lr: The maximum learning rate to be used after the warm-up period
        - total_steps: The total number of steps for training (epochs × steps_per_epoch)
        - warmup_steps: The number of steps in the warm-up phase
        - warmup_lr: The initial learning rate for warm-up, default is 0
        """
        super().__init__()
        self.base_lr = base_lr
        self.total_steps = total_steps
        self.warmup_steps = warmup_steps
        self.warmup_lr = warmup_lr
    def __call__(self, step):
        if step is None:
            step = tf.constant(0)
        step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        total_steps = tf.cast(self.total_steps, tf.float32)
        warmup_percent_done = step / warmup_steps
        learning_rate = tf.where(
            step < warmup_steps,
            self.warmup_lr + (self.base_lr - self.warmup_lr) * warmup_percent_done,
            self.base_lr * 0.5 * (1.0 + tf.cos(math.pi * (step - warmup_steps) / (total_steps - warmup_steps)))
        )
        return learning_rate
    def get_config(self):
        return {
            "base_lr": self.base_lr,
            "total_steps": self.total_steps,
            "warmup_steps": self.warmup_steps,
            "warmup_lr": self.warmup_lr,
        }
total_steps = epochs * (x_train.shape[0] // batch_size)
warmup_steps = warmup_epochs * (x_train.shape[0] // batch_size)
lr_schedule = WarmUpCosine(initial_lr, total_steps, warmup_steps)

In [9]:
class CustomWeightDecaySGD(tf.keras.optimizers.SGD):
    """
    SGD optimizer with explicit Weight Decay (Decoupled Weight Decay) 
    This optimizer, based on the standard SGD, manually applies weight 
    decay to the parameters, instead of achieving it through the L2 
    regularization term in the loss. This "decoupled weight decay"
    is in line with the idea of AdamW/SGDW and can avoid the problem 
    that weight decay is indirectly scaled by factors such as learning 
    rate and momentum. 
    """
    def __init__(self, weight_decay, **kwargs):
        """
        Parameter description:
        - weight_decay: Weight decay coefficient (usually ranging from 1e-4 to 1e-2)
        - **kwargs: Other parameters are directly passed to tf.keras.optimizers.SGD,
        such as learning_rate, momentum, nesterov, etc. 
        """
        super().__init__(**kwargs)
        self.weight_decay = weight_decay
    def apply_gradients(self, grads_and_vars, name=None, experimental_aggregate_gradients=True):
        super().apply_gradients(grads_and_vars, name, experimental_aggregate_gradients)
        for grad, var in grads_and_vars:
            if ('kernel' in var.name) and ('bn' not in var.name.lower()):
                var.assign_sub(self.weight_decay * var)
    def get_config(self):
        config = super().get_config()
        config.update({
            "weight_decay": float(self.weight_decay), 
        })
        return config
optimizer = CustomWeightDecaySGD(
    weight_decay=weight_decay,
    learning_rate=lr_schedule,
    momentum=0.9,
    nesterov=True
)

In [10]:
loss_fn=tf.keras.losses.CategoricalCrossentropy()
model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=['accuracy']
)

In [11]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [12]:
model.fit(datagen.flow(x_train, y_train_onehot,batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=epochs,
                            validation_data=(x_test, y_test_onehot),verbose=2)

Epoch 1/200
390/390 - 40s - loss: 1.5427 - accuracy: 0.4429 - val_loss: 1.9395 - val_accuracy: 0.4546 - 40s/epoch - 102ms/step
Epoch 2/200
390/390 - 22s - loss: 1.0295 - accuracy: 0.6403 - val_loss: 1.5705 - val_accuracy: 0.5667 - 22s/epoch - 56ms/step
Epoch 3/200
390/390 - 22s - loss: 0.8367 - accuracy: 0.7148 - val_loss: 1.6181 - val_accuracy: 0.5931 - 22s/epoch - 57ms/step
Epoch 4/200
390/390 - 22s - loss: 0.7372 - accuracy: 0.7498 - val_loss: 1.0389 - val_accuracy: 0.6706 - 22s/epoch - 57ms/step
Epoch 5/200
390/390 - 23s - loss: 0.6650 - accuracy: 0.7756 - val_loss: 0.9519 - val_accuracy: 0.6912 - 23s/epoch - 58ms/step
Epoch 6/200
390/390 - 23s - loss: 0.5960 - accuracy: 0.7973 - val_loss: 0.6528 - val_accuracy: 0.7832 - 23s/epoch - 58ms/step
Epoch 7/200
390/390 - 23s - loss: 0.5257 - accuracy: 0.8228 - val_loss: 0.6163 - val_accuracy: 0.7889 - 23s/epoch - 58ms/step
Epoch 8/200
390/390 - 23s - loss: 0.4828 - accuracy: 0.8332 - val_loss: 0.7178 - val_accuracy: 0.7589 - 23s/epoch - 5

In [13]:
model.save("VGG16_cifar10.h5")